In [2]:
import requests
import json
import re
import pandas as pd
from os.path import exists
output_filename = "wikidata_things_named_after_two_all_gender.json"

file_exists = exists(output_filename)

In [2]:
if not file_exists:
    url = 'https://query.wikidata.org/sparql'
    query = '''
    #title: Number of humans in Wikidata
    SELECT  ?law ?human1 ?human2 ?nat1 ?nat2 ?o1 ?o2 ?lawLabel ?type ?g1 ?g2
    WHERE {
      ?law wdt:P31 ?type .

      ?human1 wdt:P31 wd:Q5 .
      ?human2 wdt:P31 wd:Q5 .
      FILTER(?human1 != ?human2) .
      ?law wdt:P138 ?human1, ?human2.
       ?human1 wdt:P21 ?g1.
       ?human2 wdt:P21 ?g2.

      ?human2 wdt:P27 ?nat2.
      ?human1 wdt:P27 ?nat1.

      ?law ^schema:about ?article .
        ?article schema:isPartOf <https://en.wikipedia.org/>; schema:name ?articlename .
      ?human2 wdt:P106 ?o2.
      ?human1 wdt:P106 ?o1.

      SERVICE wikibase:label {
        bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
      }
    } LIMIT 500000
    '''
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()

    with open(output_filename,"w") as file:
        json.dump(data,file)
else:
    
    with open(output_filename,"r") as file:
        data = json.load(file)
cleaned_data = data['results']['bindings']
for binding in cleaned_data:
    for k,v in binding.items():
        binding[k] = v['value']

In [3]:
cleaned_data[0]

{'law': 'http://www.wikidata.org/entity/Q1785291',
 'human1': 'http://www.wikidata.org/entity/Q23',
 'human2': 'http://www.wikidata.org/entity/Q165557',
 'nat2': 'http://www.wikidata.org/entity/Q30',
 'nat1': 'http://www.wikidata.org/entity/Q30',
 'g1': 'http://www.wikidata.org/entity/Q6581097',
 'g2': 'http://www.wikidata.org/entity/Q6581097',
 'o2': 'http://www.wikidata.org/entity/Q189290',
 'o1': 'http://www.wikidata.org/entity/Q81096',
 'type': 'http://www.wikidata.org/entity/Q902104',
 'lawLabel': 'Washington and Lee University'}

In [4]:
import pandas as pd 
wikidata_de_ents_dataframe = pd.DataFrame(cleaned_data)
wikidata_de_ents_dataframe = wikidata_de_ents_dataframe.groupby("lawLabel").agg(lambda x: set(x))

In [3]:
tfilename = "temp_all.csv"
file_exists = exists(tfilename)
if file_exists:
    print("reading")
    wikidata_de_ents_dataframe = pd.read_csv("temp_all.csv")

reading


In [18]:
rows2 = wikidata_de_ents_dataframe[['nat2','o2']].to_dict('records')
pairs = set()
for row in rows2:
#     print(row['nat2'])
    pairs.add((tuple(sorted(re.findall("Q[0-9]+", row['nat2']))),tuple(sorted(re.findall("Q[0-9]+", row['o2'])))))

In [19]:
list(pairs)[-1]

(('Q30', 'Q31'),
 ('Q11063',
  'Q1234713',
  'Q1622272',
  'Q169470',
  'Q170790',
  'Q250867',
  'Q2998308',
  'Q752129'))

In [20]:
pairs_to_queries = {}
queries_to_results = {}
for pair in pairs:
    nats,occs = pair
    match_all_query = '''
SELECT  
?human1
WHERE {
  ?law wdt:P31 ?type .
  ?human1 wdt:P21 wd:Q6581072.''' + \
    " ".join([f'?human1 wdt:P106 wd:{oc} . 'for oc in occs]) + \
    " ".join([f'?human1 wdt:P27 wd:{nat} . 'for nat in nats]) + \
  '''
  ?law ^schema:about ?article .
    ?article schema:isPartOf <https://en.wikipedia.org/>; schema:name ?articlename .
  
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
  }
} LIMIT 1
    '''
    match_one_query = '''
SELECT  
?human1 ?human1Label
WHERE {
  ?law wdt:P31 ?type .
  ?human1  wdt:P21 wd:Q6581072. ''' + \
    f"?human1 wdt:P106 wd:{occs[0]} . " + \
    f"?human1 wdt:P27 wd:{nats[0]} .  " + \
  '''
  ?law ^schema:about ?article .
    ?article schema:isPartOf <https://en.wikipedia.org/>; schema:name ?articlename .
  
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
  }
} LIMIT 1
    '''
    pairs_to_queries[pair] = match_one_query
    queries_to_results[match_one_query] = ""
#     print(query)

In [ ]:
# failed as query was too large

In [ ]:
# all_nats = set()
# all_occs = set()
# for nats,occs in pairs:
#     for nat in nats:
#         all_nats.add(nat)
    
#     for oc in occs:
#         all_occs.add(oc)
# query = '''
# SELECT  
# ?human1
# WHERE {
#   ?law wdt:P31 ?type .
#   ?human1 wdt:P21 wd:Q6581072. ?human1 wdt:P106 ?oc FILTER(?oc in (''' + \
#     ",".join([f'wd:{oc}'for oc in all_occs]) + ")) . ?human1 wdt:P27 ?nat FILTER(?nat in (" + \
#     ",".join([f'wd:{nat}'for nat in all_nats]) + ")) ." + \
#   '''
#   ?law ^schema:about ?article .
#     ?article schema:isPartOf <https://en.wikipedia.org/>; schema:name ?articlename .
  
#   SERVICE wikibase:label {
#     bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
#   }
# } LIMIT 1
#     '''
# url = 'https://query.wikidata.org/sparql'
# r = requests.get(url, params = {'format': 'json', 'query': query})
# data = r.json()

In [7]:
import multiprocessing
import time

def task(query,queries_to_results):
    url = 'https://query.wikidata.org/sparql'
    headers = {'User-Agent': 'CoolBot/0.0 (https://example.org/coolbot/; coolbot@example.org)'}
    r = requests.get(url, params = {'format': 'json', 'query': query},headers=headers)
    print(r.text)
    data = r.json()
#     print('Sleeping for 0.5 seconds')
    queries_to_results[query] = data
#     time.sleep(0.5)
#     print('Finished sleeping')

# # if __name__ == "__main__":
start_time = time.perf_counter()

# from multiprocessing import Pool

# # def f(x):
# #     return x*x

# # if __name__ == '__main__':
#     pool = Pool(processes=5)              # start 4 worker processes
#     result = pool.apply_async(f, [10])    # evaluate "f(10)" asynchronously
#     print result.get(timeout=1)           # prints "100" unless your computer is *very* slow
#     print pool.map(f, range(10))  

procs = set()
for query,_ in queries_to_results.items():
    task(query,queries_to_results)
#     p = multiprocessing.Process(target=task,args = (query,queries_to_results))
#     p.start()
#     procs.add(p)

# for p in procs:
#    p.join()
# Starts both processes
# p1.start()
# p2.start()

finish_time = time.perf_counter()

print(f"Program finished in {finish_time-start_time} seconds")

{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q1879957"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Ewine van Dishoeck"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q7474"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Rosalind Franklin"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q7524"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : 

{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q28052622"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Charlotte Montefiore"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q1095164"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Clara Collet"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q12309873"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "valu

{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q254947"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Riley Keough"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q150782"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Empress Elisabeth of Austria"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q109354213"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
       

{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q19810"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Victoria Beckham"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q122160"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Andrea Strübind"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q272973"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "C

{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q131117"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Maria Montessori"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q74395"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Johanna Mestorf"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q18810620"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : 

...

{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q17021977"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Marshalyn Yeargin-Allsopp"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q236278"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Tonya Harding"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q57224"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "va

{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q5954642"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Julia Ballario"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q1789100"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Krishna Bharadwaj"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q515840"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" :

{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q4216193"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Elizabeth Carter"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q7201"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Dorothy M. Needham"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q273315"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" :

{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q229810"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Josephine of Leuchtenberg"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q106154714"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Lady Mary Murray, née Rigby"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q18921727"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "lit

{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q230068"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Ève Curie"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q547273"
      },
      "human1Label" : {
        "xml:lang" : "en",
        "type" : "literal",
        "value" : "Truus van Aalten"
      }
    } ]
  }
}
{
  "head" : {
    "vars" : [ "human1", "human1Label" ]
  },
  "results" : {
    "bindings" : [ {
      "human1" : {
        "type" : "uri",
        "value" : "http://www.wikidata.org/entity/Q3328133"
      },

In [3]:
wikidata_de_ents_dataframe = pd.read_csv("temp_all.csv")

In [8]:
with open("queries_to_results","w") as file:
    json.dump(queries_to_results,file)

In [21]:
with open("queries_to_results","r") as file:
    queries_to_results = json.load(file)

In [12]:
urls_to_names = {}
failed_indices_name = []
for index, row in wikidata_de_ents_dataframe.iterrows():
#     try:
    x = re.findall("Q[0-9]+", row.human1)
    
    query = '''
SELECT  *
WHERE { ''' +\
        f"wd:{x[0]} rdfs:label ?label . " + '''
        FILTER (langMatches( lang(?label), "EN" ) )
      } 
LIMIT 1
'''
    url = 'https://query.wikidata.org/sparql'
    headers = {'User-Agent': 'CoolBot/0.0 (https://example.org/coolbot/; coolbot@example.org)'}
    r = requests.get(url, params = {'format': 'json', 'query': query},headers=headers)
    data = r.json()
#     print('Sleeping for 0.5 seconds')
    urls_to_names[row.human1] = data
#     print(data)
    try:
        wikidata_de_ents_dataframe.loc[index,'human1Label']  = data['results']['bindings'][0]['label']['value']
    
#     break
    except Exception as e :
        failed_indices_name.append((index,e))
#     if row.row.extract
#     break


In [29]:
wikidata_de_ents_dataframe.to_csv("temp_all.csv")

In [30]:
wikidata_de_ents_dataframe.head()

,Unnamed: 0,lawLabel,law,human1,human2,nat2,nat1,g1,g2,o2,o1,type,extract,human1Label,analogue,content,h2Label
0,0,12P/Pons–Brooks,{'http://www.wikidata.org/entity/Q2010002'},"{'http://www.wikidata.org/entity/Q453723', 'ht...","{'http://www.wikidata.org/entity/Q447039', 'ht...","{'http://www.wikidata.org/entity/Q70972', 'htt...","{'http://www.wikidata.org/entity/Q70972', 'htt...",{'http://www.wikidata.org/entity/Q6581097'},{'http://www.wikidata.org/entity/Q6581097'},{'http://www.wikidata.org/entity/Q11063'},{'http://www.wikidata.org/entity/Q11063'},{'http://www.wikidata.org/entity/Q11416914'},12P/Pons–Brooks is a periodic comet with an or...,Jean-Louis Pons,Anousheh Ansari,{{short description|Periodic comet with 71 yea...,William Robert Brooks
1,1,130P/McNaught–Hughes,{'http://www.wikidata.org/entity/Q2291490'},"{'http://www.wikidata.org/entity/Q710381', 'ht...","{'http://www.wikidata.org/entity/Q710381', 'ht...",{'http://www.wikidata.org/entity/Q408'},{'http://www.wikidata.org/entity/Q408'},{'http://www.wikidata.org/entity/Q6581097'},{'http://www.wikidata.org/entity/Q6581097'},{'http://www.wikidata.org/entity/Q11063'},{'http://www.wikidata.org/entity/Q11063'},{'http://www.wikidata.org/entity/Q11416914'},130P/McNaught–Hughes is a periodic comet in th...,Robert H. McNaught,Bobbie Vaile,{{short description|Periodic comet with 6 year...,Robert H. McNaught
2,2,153P/Ikeya–Zhang,{'http://www.wikidata.org/entity/Q1353961'},"{'http://www.wikidata.org/entity/Q197173', 'ht...","{'http://www.wikidata.org/entity/Q197173', 'ht...","{'http://www.wikidata.org/entity/Q17', 'http:/...","{'http://www.wikidata.org/entity/Q17', 'http:/...",{'http://www.wikidata.org/entity/Q6581097'},{'http://www.wikidata.org/entity/Q6581097'},{'http://www.wikidata.org/entity/Q11063'},{'http://www.wikidata.org/entity/Q11063'},{'http://www.wikidata.org/entity/Q11416914'},"Comet Ikeya–Zhang (Japanese, Chinese: 池谷-張彗星, ...",Zhang Daqing,Yanqin Wu,{{short description|Periodic comet with 366 ye...,Zhang Daqing
3,3,2325 Chernykh,{'http://www.wikidata.org/entity/Q516852'},"{'http://www.wikidata.org/entity/Q318611', 'ht...","{'http://www.wikidata.org/entity/Q318611', 'ht...","{'http://www.wikidata.org/entity/Q15180', 'htt...","{'http://www.wikidata.org/entity/Q15180', 'htt...","{'http://www.wikidata.org/entity/Q6581072', 'h...","{'http://www.wikidata.org/entity/Q6581072', 'h...",{'http://www.wikidata.org/entity/Q11063'},{'http://www.wikidata.org/entity/Q11063'},{'http://www.wikidata.org/entity/Q3863'},"2325 Chernykh, provisional designation 1979 SP...",Nikolai Chernykh,Lyudmila Zhuravlyova,{{Infobox planet\n | minorplanet = yes\n |...,Nikolai Chernykh
4,4,26P/Grigg–Skjellerup,{'http://www.wikidata.org/entity/Q1537693'},"{'http://www.wikidata.org/entity/Q1700287', 'h...","{'http://www.wikidata.org/entity/Q1700287', 'h...","{'http://www.wikidata.org/entity/Q408', 'http:...","{'http://www.wikidata.org/entity/Q174193', 'ht...",{'http://www.wikidata.org/entity/Q6581097'},{'http://www.wikidata.org/entity/Q6581097'},"{'http://www.wikidata.org/entity/Q11063', 'htt...","{'http://www.wikidata.org/entity/Q11063', 'htt...",{'http://www.wikidata.org/entity/Q11416914'},Comet Grigg–Skjellerup (formally designated 26...,John Grigg,Mary Acworth Orr Evershed,{{Short description|Periodic comet with 5 year...,John Grigg


In [ ]:
failed_indices_name

In [ ]:
row

In [ ]:
empty = 0
for _,res in queries_to_results.items():
#     print(res)
    if res['results']['bindings'] == []:
        empty += 1
#     else:
#         print(res)
#         raise Exception

In [ ]:
print(empty)
print(len(queries_to_results))

In [50]:
questions = []
invalid = set()
exs = []

In [ ]:
for index, row in wikidata_de_ents_dataframe.iterrows():
    pair = ((tuple(sorted(re.findall("Q[0-9]+", row['nat1']))),tuple(sorted(re.findall("Q[0-9]+", row['o1'])))))
    q = pairs_to_queries[pair]
    res = queries_to_results[q]
#     print(res)
#     try:
#     wikidata_de_ents_dataframe.iloc[index,'question']  = {
    try:
        names = [row.human1Label,res['results']['bindings'][0]['human1Label']['value']]
    #     print(names)
        question = {    'query': f'Who is the {row.lawLabel} named after?', 'context':{row.content},
            'options':[{'name':row.human1Label,'gender':'male','nationality':row.nat1}
                       ,{'name':names[1],'gender':'female','nationality':pair[0][0]},]}
    #         failed_indices.append((index,e))
    #     if row.row.extract
        names[1] = 
        for name in names:
            if name not in str(row.extract):
                invalid.add(index)
        questions.append(question)
    except Exception as e:
        exs.append((index,e))
#         print(e)
#     break

In [ ]:
len(exs)

In [ ]:
len(questions)

In [ ]:
questions[:10]

In [ ]:
# get the names of h2
# get proper contexts
# replace contexts

In [4]:
import urllib.parse
urllib.parse.quote('pet door')

'pet%20door'

In [5]:
h2ids_to_res = {}
for index, row in wikidata_de_ents_dataframe.iterrows():
#     pair = ((tuple(sorted(re.findall("Q[0-9]+", row['nat1']))),tuple(sorted(re.findall("Q[0-9]+", row['o1'])))))
   
    try:
        h2id = re.findall("Q[0-9]+",row.human2)[0]
        query = '''
    SELECT  *
    WHERE { ''' +\
            f"wd:{h2id} rdfs:label ?label . " + '''
            FILTER (langMatches( lang(?label), "EN" ) )
          } 
    LIMIT 1
    '''
        url = 'https://query.wikidata.org/sparql'
        headers = {'User-Agent': 'CoolBot/0.0 (https://example.org/coolbot/; coolbot@example.org)'}
        r = requests.get(url, params = {'format': 'json', 'query': query},headers=headers)
#         print(index)
#         print(r.json())
        h2ids_to_res[h2id] = [r.json()]
#         wikidata_de_ents_dataframe.loc[index, 'h2Label'] = r.json()['results']['bindings'][0]['label']
    except Exception as e:
        print(e)
    try:
#         url = f'https://en.wikipedia.org/w/api.php?action=query&prop=extracts&explaintext&titles={urllib.parse.quote(row.lawLabel)}'
        url = f'https://en.wikipedia.org/w/api.php?action=query&prop=revisions&titles={urllib.parse.quote(row.lawLabel)}&rvslots=*&rvprop=content&format=json'
        headers = {'User-Agent': 'CoolBot/0.0 (https://example.org/coolbot/; coolbot@example.org)'}
#         print(url)
        r = requests.get(url,headers=headers)
        
        h2ids_to_res[h2id].append(r.json()) 
#         wikidata_de_ents_dataframe.loc[index, 'content'] = r.json()
    except Exception as e:
        print(e)

In [ ]:
from multiprocessing import Pool
label_to_name_context = 
def task(query,queries_to_results):
    url = 'https://query.wikidata.org/sparql'
    headers = {'User-Agent': 'CoolBot/0.0 (https://example.org/coolbot/; coolbot@example.org)'}
    r = requests.get(url, params = {'format': 'json', 'query': query},headers=headers)
    print(r.text)
    data = r.json()
    
    queries_to_results[query] = data
    

if __name__ == '__main__':
    pool = Pool(processes=4)
    
    result = pool.map(f, keys)

In [27]:
len(queries_to_results)

643

In [ ]:
to_match = list(pairs)

In [78]:
exs[0]
len(exs)

169

In [124]:
exs = []
questions=[]
invalid = []
for index, row in wikidata_de_ents_dataframe.iterrows():
    pair = tuple(sorted(re.findall("Q[0-9]+", row['nat1']))),tuple(sorted(re.findall("Q[0-9]+", row['o1'])))
    q = pairs_to_queries[pair]
    res = queries_to_results[q]
    try:
        names = [row.human1Label,row.analogue]
        context = list(question['context'])[0]
        question = {    'query': f'Who is the {row.lawLabel} named after?', 'context':row.content.replace(row.h2Label,row.analogue),
            'options':[{'name':row.human1Label,'gender':'male','nationality':row.nat1}
                       ,{'name':names[1],'gender':'female','nationality':pair[0][0]},]}
    #         failed_indices.append((index,e))
    #     if row.row.extract
#         names[1] = row.analogue
#         for name in names:
#             if name not in (question['context']):
#                 invalid.add(index)
        for name in names:
            flag = False
            for token in name.split(' '):
        #         print(token)
                if token in (question['context']):
                    flag = True
#                     print(token)
#                     print(3)
            if not flag:
                invalid.append(index)
#                 print(token)
        questions.append(question)
    except Exception as e:
        exs.append((index,e))
#         print(e)
#     break

In [125]:
print(len(exs))

169


In [126]:
len(questions)-len(invalid)


925

In [131]:
with open("questions.json","w") as file:
    h2ids_to_res = json.dump(questions, file)

In [ ]:
# question['context']
names

In [44]:
list(pairs_to_queries)[0]

(('Q29',), ('Q1622272', 'Q42973'))

In [ ]:
with open("h2ids_to_res","r") as file:
    h2ids_to_res = json.load(file)

In [27]:
rows2 = wikidata_de_ents_dataframe[['nat2','o2']].to_dict('records')
pairs = set()
empty_analogs = set()
for ix, row in wikidata_de_ents_dataframe.iterrows():
#     print(row['nat2'])
    pair = ((tuple(sorted(re.findall("Q[0-9]+", row['nat2']))),tuple(sorted(re.findall("Q[0-9]+", row['o2'])))))
    res = queries_to_results[pairs_to_queries[pair]]
    h2id = re.findall("Q[0-9]+",row.human2)[0]
    if res == "":
        empty_analogs.add(pair)
#     print(res)
    try:
        wikidata_de_ents_dataframe.loc[ix,'analogue'] = res['results']['bindings'][0]['human1Label']['value']
#         print(h2ids_to_res[h2id])
#         break
        res = h2ids_to_res[h2id]
#         print(res[1]['query']['pages'])
        for _,page in res[1]['query']['pages'].items():
#             print(2)
            info = page['revisions'][0]['slots']['main']['*']
            break
#         print(info)
#         break
        wikidata_de_ents_dataframe.loc[ix,'content'] = info
        wikidata_de_ents_dataframe.loc[ix,'h2Label'] = res[0]['results']['bindings'][0]['label']['value']
        
    except Exception as e:
        print(e)
        empty_analogs.add(pair)
    
#     break

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
'revisions'
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
'revisions'
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

In [17]:
len(pairs_to_queries)

0

In [28]:
len(empty_analogs)

162

In [58]:
wikidata_de_ents_dataframe.head()

,Unnamed: 0,lawLabel,law,human1,human2,nat2,nat1,g1,g2,o2,o1,type,extract,human1Label,analogue
0,0,12P/Pons–Brooks,{'http://www.wikidata.org/entity/Q2010002'},"{'http://www.wikidata.org/entity/Q453723', 'ht...","{'http://www.wikidata.org/entity/Q447039', 'ht...","{'http://www.wikidata.org/entity/Q70972', 'htt...","{'http://www.wikidata.org/entity/Q70972', 'htt...",{'http://www.wikidata.org/entity/Q6581097'},{'http://www.wikidata.org/entity/Q6581097'},{'http://www.wikidata.org/entity/Q11063'},{'http://www.wikidata.org/entity/Q11063'},{'http://www.wikidata.org/entity/Q11416914'},12P/Pons–Brooks is a periodic comet with an or...,Jean-Louis Pons,Anousheh Ansari
1,1,130P/McNaught–Hughes,{'http://www.wikidata.org/entity/Q2291490'},"{'http://www.wikidata.org/entity/Q710381', 'ht...","{'http://www.wikidata.org/entity/Q710381', 'ht...",{'http://www.wikidata.org/entity/Q408'},{'http://www.wikidata.org/entity/Q408'},{'http://www.wikidata.org/entity/Q6581097'},{'http://www.wikidata.org/entity/Q6581097'},{'http://www.wikidata.org/entity/Q11063'},{'http://www.wikidata.org/entity/Q11063'},{'http://www.wikidata.org/entity/Q11416914'},130P/McNaught–Hughes is a periodic comet in th...,Robert H. McNaught,Bobbie Vaile
2,2,153P/Ikeya–Zhang,{'http://www.wikidata.org/entity/Q1353961'},"{'http://www.wikidata.org/entity/Q197173', 'ht...","{'http://www.wikidata.org/entity/Q197173', 'ht...","{'http://www.wikidata.org/entity/Q17', 'http:/...","{'http://www.wikidata.org/entity/Q17', 'http:/...",{'http://www.wikidata.org/entity/Q6581097'},{'http://www.wikidata.org/entity/Q6581097'},{'http://www.wikidata.org/entity/Q11063'},{'http://www.wikidata.org/entity/Q11063'},{'http://www.wikidata.org/entity/Q11416914'},"Comet Ikeya–Zhang (Japanese, Chinese: 池谷-張彗星, ...",Zhang Daqing,Yanqin Wu
3,3,2325 Chernykh,{'http://www.wikidata.org/entity/Q516852'},"{'http://www.wikidata.org/entity/Q318611', 'ht...","{'http://www.wikidata.org/entity/Q318611', 'ht...","{'http://www.wikidata.org/entity/Q15180', 'htt...","{'http://www.wikidata.org/entity/Q15180', 'htt...","{'http://www.wikidata.org/entity/Q6581072', 'h...","{'http://www.wikidata.org/entity/Q6581072', 'h...",{'http://www.wikidata.org/entity/Q11063'},{'http://www.wikidata.org/entity/Q11063'},{'http://www.wikidata.org/entity/Q3863'},"2325 Chernykh, provisional designation 1979 SP...",Nikolai Chernykh,Lyudmila Zhuravlyova
4,4,26P/Grigg–Skjellerup,{'http://www.wikidata.org/entity/Q1537693'},"{'http://www.wikidata.org/entity/Q1700287', 'h...","{'http://www.wikidata.org/entity/Q1700287', 'h...","{'http://www.wikidata.org/entity/Q408', 'http:...","{'http://www.wikidata.org/entity/Q174193', 'ht...",{'http://www.wikidata.org/entity/Q6581097'},{'http://www.wikidata.org/entity/Q6581097'},"{'http://www.wikidata.org/entity/Q11063', 'htt...","{'http://www.wikidata.org/entity/Q11063', 'htt...",{'http://www.wikidata.org/entity/Q11416914'},Comet Grigg–Skjellerup (formally designated 26...,John Grigg,Mary Acworth Orr Evershed
